## **Rating Product & Sorting Reviews in Amazon**

### **İş Problemi**

E-ticaretteki en önemli problemlerden bir tanesi ürünlere satış
sonrası verilen puanların doğru şekilde hesaplanmasıdır. Bu
problemin çözümü e-ticaret sitesi için daha fazla müşteri
memnuniyeti sağlamak, satıcılar için ürünün öne çıkması ve satın
alanlar için sorunsuz bir alışveriş deneyimi demektir. Bir diğer
problem ise ürünlere verilen yorumların doğru bir şekilde
sıralanması olarak karşımıza çıkmaktadır. Yanıltıcı yorumların öne
çıkması ürünün satışını doğrudan etkileyeceğinden dolayı hem
maddi kayıp hem de müşteri kaybına neden olacaktır. Bu 2 temel
problemin çözümünde e-ticaret sitesi ve satıcılar satışlarını
arttırırken müşteriler ise satın alma yolculuğunu sorunsuz olarak
tamamlayacaktır.


### **Veri Seti Hikayesi**

Amazon ürün verilerini içeren bu veri seti ürün kategorileri ile çeşitli metadataları içermektedir. Elektronik kategorisindeki en fazla yorum alan ürünün kullanıcı puanları ve yorumları vardır.

* **reviewerID :** Kullanıcı ID’si

* **asin** Ürün ID’si

* **reviewerName :** Kullanıcı Adı

* **helpful :** Faydalı değerlendirme derecesi

* **reviewText :** Değerlendirme

* **overall :** Ürün rating’i

* **summary :** Değerlendirme özeti

* **unixReviewTime :** Değerlendirme zamanı

* **reviewTime :** Değerlendirme zamanı Raw

* **day_diff :** Değerlendirmeden itibaren geçen gün sayısı

* **helpful_yes :** Değerlendirmenin faydalı bulunma sayısı

* **total_vote :** Değerlendirmeye verilen oy sayısı

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

import datetime as dt
import math
import scipy.stats as st

In [3]:
df_ = pd.read_csv("/content/gdrive/MyDrive/DSMLBC10/week_5 (27.10.22-02.11.22)/project/amazon_review.csv")

In [4]:
df = df_.copy()
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [6]:
df.isnull().sum()

reviewerID        0
asin              0
reviewerName      1
helpful           0
reviewText        1
overall           0
summary           0
unixReviewTime    0
reviewTime        0
day_diff          0
helpful_yes       0
total_vote        0
dtype: int64

In [7]:
df["asin"].value_counts() #tek bir ürün var.

B007WTAJTO    4915
Name: asin, dtype: int64

## **PROJE GÖREVLERİ**

####**Görev 1:** Average Rating’i güncel yorumlara göre hesaplayınız ve var olan average rating ile kıyaslayınız.

Paylaşılan veri setinde kullanıcılar bir ürüne puanlar vermiş ve yorumlar yapmıştır. Bu görevde amacımız verilen puanları tarihe göre
ağırlıklandırarak değerlendirmek. İlk ortalama puan ile elde edilecek tarihe göre ağırlıklı puanın karşılaştırılması gerekmektedir.

#### **Adım 1:** Ürünün ortalama puanını hesaplayınız.



In [8]:
df["overall"].mean()

4.587589013224822

#### **Adım 2:** Tarihe göre ağırlıklı puan ortalamasını hesaplayınız.


• reviewTime değişkenini tarih değişkeni olarak tanıtmanız

• reviewTime'ın max değerini current_date olarak kabul etmeniz

• her bir puan-yorum tarihi ile current_date'in farkını gün cinsinden ifade ederek yeni değişken oluşturmanız ve gün cinsinden ifade edilen
değişkeni quantile fonksiyonu ile 4'e bölüp (3 çeyrek verilirse 4 parça çıkar) çeyrekliklerden gelen değerlere göre ağırlıklandırma yapmanız
gerekir.

 Örneğin q1 = 12 ise ağırlıklandırırken 12 günden az süre önce yapılan yorumların ortalamasını alıp bunlara yüksek ağırlık vermek gibi.

In [9]:
df["reviewTime"] = df["reviewTime"].apply(pd.to_datetime)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   reviewerID      4915 non-null   object        
 1   asin            4915 non-null   object        
 2   reviewerName    4914 non-null   object        
 3   helpful         4915 non-null   object        
 4   reviewText      4914 non-null   object        
 5   overall         4915 non-null   float64       
 6   summary         4915 non-null   object        
 7   unixReviewTime  4915 non-null   int64         
 8   reviewTime      4915 non-null   datetime64[ns]
 9   day_diff        4915 non-null   int64         
 10  helpful_yes     4915 non-null   int64         
 11  total_vote      4915 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 460.9+ KB


In [10]:
current_date = df["reviewTime"].max()
current_date

Timestamp('2014-12-07 00:00:00')

In [11]:
df["days"] = (current_date - df["reviewTime"]).dt.days
df["days"].head()

0    137
1    408
2    714
3    381
4    512
Name: days, dtype: int64

In [12]:
a = df["days"].quantile(0.25)
b = df["days"].quantile(0.50)
c = df["days"].quantile(0.75)

print(a,b,c)

280.0 430.0 600.0


In [13]:
def time_based_weighted_average(dataframe, w1=30, w2=24, w3=26, w4=20):
    return dataframe.loc[df["days"] <= a, "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["days"] > a) & (dataframe["days"] <= b), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["days"] > b) & (dataframe["days"] <= c), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["days"] > c), "overall"].mean() * w4 / 100

time_based_weighted_average(df)

4.599294353300253

#### **Adım 3:** Ağırlıklandırılmış puanlamada her bir zaman diliminin ortalamasını karşılaştırıp yorumlayınız.


In [14]:
df.loc[df["days"] <= a, "overall"].mean() * 30 / 100

1.40873786407767

In [15]:
df.loc[(df["days"] > a) & (df["days"] <= b), "overall"].mean() * 24 / 100

1.1126737530662307

In [16]:
df.loc[(df["days"] > b) & (df["days"] <= c), "overall"].mean() * 26 / 100

1.1886319218241044

In [17]:
df.loc[(df["days"] > c), "overall"].mean() * 20 / 100

0.8892508143322475

#### **Görev 2 :** Ürün için ürün detay sayfasında görüntülenecek 20 review'i belirleyiniz. 

#### **Adım 1 :** helpful_no değişkenini üretiniz.

• total_vote bir yoruma verilen toplam up-down sayısıdır.

• up, helpful demektir.

• Veri setinde helpful_no değişkeni yoktur, var olan değişkenler üzerinden üretilmesi gerekmektedir.

• Toplam oy sayısından (total_vote) yararlı oy sayısı (helpful_yes) çıkarılarak yararlı bulunmayan oy sayılarını (helpful_no) bulunuz.


In [18]:
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0,512,0


#### **Adım 2 :** score_pos_neg_diff, score_average_rating ve wilson_lower_bound skorlarını hesaplayıp veriye ekleyiniz.

• score_pos_neg_diff, score_average_rating ve wilson_lower_bound skorlarını hesaplayabilmek için score_pos_neg_diff,
score_average_rating ve wilson_lower_bound fonksiyonlarını tanımlayınız.

• score_pos_neg_diff'a göre skorlar oluşturunuz. Ardından; df içerisinde score_pos_neg_diff ismiyle kaydediniz.

• score_average_rating'a göre skorlar oluşturunuz. Ardından; df içerisinde score_average_rating ismiyle kaydediniz.

• wilson_lower_bound'a göre skorlar oluşturunuz. Ardından; df içerisinde wilson_lower_bound ismiyle kaydediniz.

In [19]:
up = df["helpful_yes"].tolist()
down = df["helpful_no"].tolist()

def score_up_down_diff(up, down):
  return up - down 

df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)

def score_average_rating(up, down):
  if up + down == 0:
    return 0
  return up / (up + down)

df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)

def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)


df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137,0,0,0.0,0.0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408,0,0,0.0,0.0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714,0,0,0.0,0.0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381,0,0,0.0,0.0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0,512,0,0,0.0,0.0


#### **Adım 3 :** 20 Yorumu belirleyiniz ve sonuçları Yorumlayınız.

• wilson_lower_bound'a göre ilk 20 yorumu belirleyip sıralayınız.

• Sonuçları yorumlayınız.

In [20]:
df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.0,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,701,68,1884,0.966337,0.957544
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.0,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,802,77,1351,0.948837,0.936519
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.0,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,578,126,1442,0.925620,0.912139
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.0,"Warning, read this!",1346544000,2012-02-09,1033,422,495,1032,73,349,0.852525,0.818577
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.0,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,157,4,41,0.918367,0.808109
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.0,I own it,1393545600,2014-02-28,283,60,68,282,8,52,0.882353,0.784651
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.0,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,776,27,85,0.805755,0.732136
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.0,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,648,3,19,0.880000,0.700442
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.0,Awesome Card!,1339200000,2012-09-06,823,51,65,822,14,37,0.784615,0.670334
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.0,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,806,27,55,0.752294,0.663595


45-4, 22-3, 7-0 gibi az yorumlar olsa da pozitif yorumun fazlalığı üst sıralarda olmasına sebep olmuştur. 